In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from nn_with_hidden_layer import *

In [2]:
# Data
df = pd.read_csv('credit.csv')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
# Data preprocessing
df = df.dropna()
df.replace({"Loan_Status" : {'N':0, 'Y':1}}, inplace=True)

# Labelling
df = df.replace(to_replace='3+', value=4)
df.replace({'Married':{'No':0, 'Yes':1}, 'Gender':{'Male':1, 'Female':0}, 'Self_Employed':{'No':0, 'Yes':1},
                      'Property_Area':{"Rural":0, 'Semiurban':1, 'Urban':2}, 'Education':{'Graduate':1, 'Not Graduate':0}}, inplace=True)

x = df.drop(columns=['Loan_ID', "Loan_Status"] , axis=1)
y = df.Loan_Status # label

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x,y , test_size=0.2, stratify=y, random_state=30)

In [24]:
x_train = normalize(x_train.to_numpy(dtype=int))
x_test = normalize(x_test.to_numpy(dtype=int))
y_train = y_train.to_numpy(dtype=int)
y_test = y_test.to_numpy(dtype=int)

In [128]:
# Neural Network with hidden units
def normalize(x):
    mean = np.mean(x)
    std_dev = np.std(x)
    z = (x - mean) / std_dev
    return z

def sigmoid(z):
    z_clipped = np.clip(z, -500, 500)  # Clip values to the range [-500, 500]
    return 1 / (1 + np.exp(-z_clipped))

def sigmoid_derivative(z):
    gz = sigmoid(z)
    return gz * (1 - gz)

def linear(x,w,b):
    return (w * x) + b

def relu(z):
    return np.maximum(0,z)

def relu_derivative(z):
     return np.where(z > 0, 1, 0)

def tanh_derivative(z):
    return 1 - (tanh(z) ** 2)

def leaky_relu(z):
    return np.maximum(0.01 * z, z)

def tanh(z):
    return np.tanh(z)

def loss_sigmoid(y_pred, y):
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.mean(- (y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))) 
    return loss

def init_params(x, hidden_units, output_units):
    input_units = x.shape[1]
    w1 = np.random.randn(hidden_units, input_units)
    b1 = np.zeros((hidden_units,  output_units))
    w2 = np.random.randn(output_units, hidden_units)
    b2 = np.zeros((output_units, 1))
    
    return w1, b1, w2, b2
    
def back_prop(x,y,a1,a2,w2,z1):
    m = x.shape[1]
    
    dz2 = a2 - y # y stacked horizontally
    dw2 = np.dot(dz2 ,a1.T) / m
    db2 = (np.sum(dz2, axis=1, keepdims=True)) / m
    dz1 = np.dot(w2.T,dz2) * relu_derivative(z1)
    dw1 = np.dot(dz1 ,x) / m
    db1 = (np.sum(dz1, axis=1, keepdims=True)) / m
    
    return dw1, db1, dw2, db2

def forward_prop(x, y, hidden_units, learning_rate, epochs):
    w1, b1, w2, b2 = init_params(x, hidden_units, output_units=1)
    
    for i in range(epochs+1):
        z1 = np.dot(w1,x.T) + b1
        a1 = relu(z1)
        z2 = np.dot(w2, a1) + b2
        a2 = sigmoid(z2)
        
        dw1, db1, dw2, db2 = back_prop(x, y, a1, a2, w2, z1)
        
        w1 -= learning_rate * dw1
        b1 -= learning_rate * db1
        w2 -= learning_rate * dw2 
        b2 -= learning_rate * db2
        
        if i % math.ceil(epochs/10) == 0:
            print(f"Iteration: {i} Loss: {loss_sigmoid(a2, y)}")
    
    return w1, b1, w2, b2

def predict(x, w1, b1, w2, b2):
    z1 = np.dot(x, w1.T) + b1.reshape(1,w1.shape[0])
    a1 = relu(z1)
    z2 = np.dot(a1, w2.T) + b2
    a2 = sigmoid(z2)
    
    y_hat = (a2 > 0.5).astype(int)
    
    return y_hat.T

def accuracy(yhat, y):
    return  (np.sum(yhat == y) / y.shape[0]) * 100
    
fit = forward_prop

In [ ]:
w1, b1, w2, b2 = fit(x_train, y_train, hidden_units=100, learning_rate=0.01, epochs=6500)

Iteration: 0 Loss: 0.7972796754393565
Iteration: 650 Loss: 0.5846077212312092
Iteration: 1300 Loss: 0.5738501809732334
Iteration: 1950 Loss: 0.5689998506764584
Iteration: 2600 Loss: 0.5649676938779652
Iteration: 3250 Loss: 0.5528173798767012
Iteration: 3900 Loss: 0.5490303810437179
Iteration: 4550 Loss: 0.615824169563417
Iteration: 5200 Loss: 0.5313046821565992
Iteration: 5850 Loss: 0.5360231144141944
Iteration: 6500 Loss: 0.5262618427535856


In [131]:
yhat = predict(x_train,w1,b1,w2,b2)
accuracy(yhat, y_train)

73.17708333333334

In [132]:
yhat_test = predict(x_test,w1,b1,w2,b2)
accuracy(yhat_test, y_test)

67.70833333333334